<a href="https://colab.research.google.com/github/Ritviks21/Silicon-Sentinel/blob/main/Silicon_Sentinel_Training_ipynb_just_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Install Libraries

In [ ]:
!pip install ultralytics opencv-python n

Final Data Generation Script

In [ ]:
import os
import cv2
import numpy as np
import random
import shutil
import math

# --- Configuration ---
DATASET_PATH = 'ultimate_wafer_dataset'
NUM_IMAGES = 2500
IMG_WIDTH = 640
IMG_HEIGHT = 640
defect_classes = {"scratch": 0, "particle": 1, "blob": 2}

# --- Helper Functions ---

def create_base_wafer():
    """
    Creates a more realistic, textured silicon wafer background with variations
    to prevent the model from overfitting to a single background pattern.
    """
    bg_type = random.choice(['smooth', 'grainy', 'gradient'])
    base_color = random.randint(45, 60)
    image = np.full((IMG_HEIGHT, IMG_WIDTH, 3), base_color, dtype=np.uint8)

    if bg_type == 'grainy':
        noise = np.random.normal(0, 3, (IMG_HEIGHT, IMG_WIDTH, 3)).astype(np.int16)
        image = np.clip(image.astype(np.int16) + noise, 0, 255).astype(np.uint8)
        image = cv2.GaussianBlur(image, (3, 3), 0)
    elif bg_type == 'gradient':
        end_color = base_color - random.randint(10, 20)
        c = np.linspace(base_color, end_color, IMG_WIDTH, dtype=np.uint8)
        image[:, :, :] = c[np.newaxis, :, np.newaxis]

    # Add a subtle color tint to mimic silicon
    tint_color = np.array([1.0, 1.0, random.uniform(1.0, 1.05)])
    image = np.clip(image * tint_color, 0, 255).astype(np.uint8)
    return image

def add_advanced_scratch(image):
    """
    Generates straight, curved, or wavy scratches to teach the model to
    differentiate from simple background lines.
    """
    start_x, start_y = random.randint(50, IMG_WIDTH - 50), random.randint(50, IMG_HEIGHT - 50)
    thickness, color = random.randint(1, 2), (random.randint(220, 255),) * 3
    points = [(start_x, start_y)]
    scratch_type = random.choice(['straight', 'curved', 'wavy'])
    if scratch_type == 'straight':
        end_x, end_y = start_x + random.randint(-200, 200), start_y + random.randint(-200, 200)
        points.append((end_x, end_y))
    elif scratch_type == 'curved':
        end_x, end_y = start_x + random.randint(-200, 200), start_y + random.randint(-200, 200)
        ctrl_x = (start_x + end_x) // 2 + random.randint(-50, 50)
        ctrl_y = (start_y + end_y) // 2 + random.randint(-50, 50)
        for i in range(1, 21):
            t = i / 20.0
            x = int((1 - t)**2 * start_x + 2 * (1 - t) * t * ctrl_x + t**2 * end_x)
            y = int((1 - t)**2 * start_y + 2 * (1 - t) * t * ctrl_y + t**2 * end_y)
            points.append((x,y))
    elif scratch_type == 'wavy':
        length, angle = random.randint(100, 300), random.uniform(0, 2 * math.pi)
        freq, amp = random.uniform(0.02, 0.05), random.uniform(5, 15)
        for i in range(length):
            x = start_x + int(i * math.cos(angle) - amp * math.sin(i * freq) * math.sin(angle))
            y = start_y + int(i * math.sin(angle) + amp * math.sin(i * freq) * math.cos(angle))
            points.append((x,y))

    for i in range(len(points) - 1):
        if 0 <= points[i][0] < IMG_WIDTH and 0 <= points[i][1] < IMG_HEIGHT and \
           0 <= points[i+1][0] < IMG_WIDTH and 0 <= points[i+1][1] < IMG_HEIGHT:
            cv2.line(image, points[i], points[i+1], color, thickness)

    all_x = [p[0] for p in points]; all_y = [p[1] for p in points]
    x_min, x_max = min(all_x), max(all_x)
    y_min, y_max = min(all_y), max(all_y)
    x_center, y_center = (x_min + x_max) / 2, (y_min + y_max) / 2
    width, height = x_max - x_min, y_max - y_min

    return image, [defect_classes["scratch"], x_center / IMG_WIDTH, y_center / IMG_HEIGHT, width / IMG_WIDTH, height / IMG_HEIGHT]


def add_distinct_particles(image):
    """
    Adds small, sharp, bright 'dust' specks to solve class confusion and
    improve detection of small objects.
    """
    num_particles = random.randint(1, 15)
    all_x, all_y = [], []
    for _ in range(num_particles):
        px, py = random.randint(20, IMG_WIDTH - 20), random.randint(20, IMG_HEIGHT - 20)
        radius = random.randint(1, 2)
        color = (random.randint(230, 255),) * 3
        cv2.circle(image, (px, py), radius, color, -1)
        all_x.extend([px - radius, px + radius]); all_y.extend([py - radius, py + radius])

    x_min, x_max, y_min, y_max = min(all_x), max(all_x), min(all_y), max(all_y)
    x_center, y_center = (x_min + x_max) / 2, (y_min + y_max) / 2
    width, height = x_max - x_min, y_max - y_min
    return image, [defect_classes["particle"], x_center / IMG_WIDTH, y_center / IMG_HEIGHT, width / IMG_WIDTH, height / IMG_HEIGHT]

def add_distinct_blob(image):
    """
    Adds a larger, irregular, transparent 'smudge' to solve class confusion.
    """
    overlay, output = image.copy(), image.copy()
    center_x, center_y = random.randint(150, IMG_WIDTH - 150), random.randint(150, IMG_HEIGHT - 150)

    num_points = random.randint(5, 10)
    angles = sorted([random.uniform(0, 2 * math.pi) for _ in range(num_points)])
    points = []
    for angle in angles:
        radius = random.uniform(40, 80)
        x = center_x + int(radius * math.cos(angle))
        y = center_y + int(radius * math.sin(angle))
        points.append([x, y])
    points = np.array(points, np.int32)

    color = (random.randint(80, 100),) * 3
    cv2.fillPoly(overlay, [points], color)
    overlay = cv2.GaussianBlur(overlay, (21, 21), 0)

    transparency = random.uniform(0.2, 0.5)
    cv2.addWeighted(overlay, transparency, output, 1 - transparency, 0, image)

    all_x, all_y = points[:, 0], points[:, 1]
    x_min, x_max, y_min, y_max = min(all_x), max(all_x), min(all_y), max(all_y)
    x_center, y_center = (x_min + x_max) / 2, (y_min + y_max) / 2
    width, height = x_max - x_min, y_max - y_min
    return image, [defect_classes["blob"], x_center / IMG_WIDTH, y_center / IMG_HEIGHT, width / IMG_WIDTH, height / IMG_HEIGHT]

# --- Main Generation Function ---
def generate_ultimate_dataset():
    images_path = os.path.join(DATASET_PATH, 'images')
    labels_path = os.path.join(DATASET_PATH, 'labels')
    if os.path.exists(DATASET_PATH): shutil.rmtree(DATASET_PATH)
    os.makedirs(images_path, exist_ok=True); os.makedirs(labels_path, exist_ok=True)
    print(f"Directories created at: {os.path.abspath(DATASET_PATH)}")
    for i in range(NUM_IMAGES):
        wafer_image, annotations = create_base_wafer(), []
        if random.random() >= 0.25: # 25% of images will be clean "negative" samples
            for _ in range(random.randint(1, 4)):
                defect_type = random.choice(list(defect_classes.keys()))
                if defect_type == "scratch": wafer_image, ann = add_advanced_scratch(wafer_image)
                elif defect_type == "particle": wafer_image, ann = add_distinct_particles(wafer_image)
                else: wafer_image, ann = add_distinct_blob(wafer_image)
                if ann and ann[3] > 0 and ann[4] > 0: annotations.append(ann)
        img_name, label_name = f'wafer_{i:04d}.png', f'wafer_{i:04d}.txt'
        cv2.imwrite(os.path.join(images_path, img_name), wafer_image)
        with open(os.path.join(labels_path, label_name), 'w') as f:
            for ann in annotations: f.write(f"{ann[0]} {ann[1]} {ann[2]} {ann[3]} {ann[4]}\n")
        if (i + 1) % 250 == 0: print(f"  -> Generated {i + 1}/{NUM_IMAGES} images...")
    print(f"\n✅ Ultimate dataset generation complete!")

generate_ultimate_dataset()

Split the Ultimate Dataset

In [ ]:
import os
import random
import shutil

# --- Configuration ---
DATA_SOURCE_DIR = 'ultimate_wafer_dataset'
OUTPUT_DIR = 'ultimate_data_for_training'
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2

# --- Splitting Logic ---
images_source_path = os.path.join(DATA_SOURCE_DIR, 'images')
all_images = [f for f in os.listdir(images_source_path) if f.endswith('.png')]
random.shuffle(all_images)

total_images = len(all_images)
train_end = int(total_images * TRAIN_RATIO)
val_end = train_end + int(total_images * VAL_RATIO)

train_files = all_images[:train_end]
val_files = all_images[train_end:val_end]
test_files = all_images[val_end:]

def copy_files(file_list, set_name):
    dest_images_path = os.path.join(OUTPUT_DIR, set_name, 'images')
    dest_labels_path = os.path.join(OUTPUT_DIR, set_name, 'labels')
    os.makedirs(dest_images_path, exist_ok=True)
    os.makedirs(dest_labels_path, exist_ok=True)
    source_images_path = os.path.join(DATA_SOURCE_DIR, 'images')
    source_labels_path = os.path.join(DATA_SOURCE_DIR, 'labels')
    for filename in file_list:
        base_name = os.path.splitext(filename)[0]
        shutil.copy(os.path.join(source_images_path, filename), os.path.join(dest_images_path, filename))
        shutil.copy(os.path.join(source_labels_path, f'{base_name}.txt'), os.path.join(dest_labels_path, f'{base_name}.txt'))
    print(f"  -> Copied {len(file_list)} files to the '{set_name}' set.")

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
copy_files(train_files, 'train')
copy_files(val_files, 'val')
copy_files(test_files, 'test')

# --- Create YAML File ---
yaml_content = f"""
path: {os.path.abspath(OUTPUT_DIR)}
train: train/images
val: val/images
test: test/images

nc: 3
names: ['scratch', 'particle', 'blob']
"""
with open(os.path.join(OUTPUT_DIR, 'data.yaml'), 'w') as f:
    f.write(yaml_content)

print(f"\n✅ Ultimate dataset successfully split.")

Train the Final Model

In [ ]:
from ultralytics import YOLO

# --- Load the upgraded YOLOv8s model ---
model = YOLO('yolov8s.pt')

# --- Define the Google Drive save path ---
drive_project_path = '/content/drive/MyDrive/wafer_project/wafer_final_runs'

# --- Start the final training ---
print("🚀 Starting the FINAL model training with YOLOv8s, full augmentation, and 75 epochs...")
results = model.train(
    data='ultimate_data_for_training/data.yaml',
    epochs=75,         # Increased for better learning
    imgsz=640,
    degrees=15,        # Data augmentation
    translate=0.1,
    scale=0.1,
    fliplr=0.5,
    project=drive_project_path,
    name='Silicon_Sentinel_v1.0' # Final model name
)

print("✅ FINAL training complete!")
print(f"Model and results are permanently saved in your Google Drive at: {results.save_dir}")

Final Evaluation

In [ ]:
import os
import glob
from ultralytics import YOLO
from IPython.display import Image, display

# --- Unzip the test images ---
print("Unzipping test images...")
!unzip -o test_images.zip -d test_images_unzipped
print("✅ Unzip complete.")

# --- Final Model Evaluation ---

# 1. Path to our ultimate trained model in your Google Drive.
model_path = '/content/drive/MyDrive/wafer_project/wafer_final_runs/Silicon_Sentinel_v1.0/weights/best.pt'

# 2. Path to the test images folder.
test_images_folder = 'test_images_unzipped/test_images'

# 3. Load our final, powerful model.
print(f"\nLoading Silicon Sentinel from: {model_path}")
model = YOLO(model_path)
print("✅ Final model loaded successfully.")

# 4. Run prediction with our strict 0.5 confidence threshold.
print("\n🚀 Running FINAL predictions...")
results = model.predict(
    source=test_images_folder,
    save=True,
    conf=0.5
)
print("✅ Final predictions complete!")

# 5. Display the definitive results.
prediction_output_dir = results[0].save_dir
result_images = sorted(glob.glob(os.path.join(prediction_output_dir, '*.jpg')))

if not result_images:
    print("\nCould not find any result images.")
else:
    print("\n--- ✅ SILICON SENTINEL: FINAL RESULTS ---")
    for img_path in result_images:
        print(f"\nResult for: {os.path.basename(img_path)}")
        display(Image(filename=img_path))

In [ ]:
import os
import glob
from ultralytics import YOLO
from IPython.display import Image, display

# --- Final Showcase Generation ---
print("This is the final run. We will generate several sets of results for you to choose from.")

# The path to the ULTIMATE model, which is safely in your Google Drive
model_path = '/content/drive/MyDrive/wafer_project/wafer_final_runs/Silicon_Sentinel_v1.0/weights/best.pt'
test_images_folder = 'test_images_unzipped/test_images'

if not os.path.exists(model_path):
    print(f"❌ ERROR: The final model was not found in your Google Drive at {model_path}")
else:
    model = YOLO(model_path)

    # We will test at three different confidence levels
    confidence_levels = [0.40, 0.25, 0.15]

    for conf in confidence_levels:
        print(f"\n🚀 Running prediction with confidence threshold = {conf}...")
        model.predict(
            source=test_images_folder,
            save=True,
            conf=conf,
            name=f"predict_at_{int(conf*100)}" # Saves to a unique folder like 'predict_at_25'
        )
        print(f"✅ Prediction run for conf={conf} complete!")

print("\n--- All showcase images have been generated. ---")
print("Please check the folders in 'runs/detect/' (e.g., 'predict_at_40', 'predict_at_25') to find your images.")